<center><h4>McGill University ECSE 415 Introduction to Computer Vision F23 - Final Project</h4>
<h1><font color='green'>ORION: A YOLO-based Object Tracking and Statistical Dashcam System</font></h1>
<h4>Alex Wei 260981800 / Zhanyue Zhang 260944809</h4></center>


[![GitHub](https://badges.aleen42.com/src/github.svg)](https://github.com/ThermalG/Computer-Vision)

&nbsp;
## *¡README!*
1. The project was developed with IntelliJ IDEA 2023.2, Python 3.11 and CUDA Toolkit 12.3. In local venv, please be advised that the GUI window will not auto pop up upon first run, so pay attention to your taskbar / dock. Also you might have to [configure the environment](https://www.jamesbowley.co.uk/qmd/opencv_cuda_python_windows.html) carefully in order to maximize the performance. Should you run in a cloud env like Colab, please first ensure appropriate adaptations, including but not limited to:
    - Remove PyQt5 snippets and set `path` to your directory with intended video(s);
    - Uncomment the first cell before running;
    - Optional: select GPU in runtime type;
    - Optional: uninstall default cv2 and replace with your own built in CMake.
2. The `PATH` variable below denotes where I store input / output videos (debugging & visualization) and other models. You can choose whether or not you want a video ouput or simply the statistics prints on console.
3. Only car objects (coco index 2) are considered as "cars"; (motor)cyclists are excluded from pedestrian count.

&nbsp;
## Change Log / To Dos
> \* UNFINISHED TASKS (# PRIORITY)

**v0.1 20/11/2023 Prototype**

**v0.2 23/11/2023**
1. Bugfix: Solved `WARNING ⚠️ 'source' is missing. Using 'source=C:\Users\...` upon start.
2. Bugfix: Solved being unable to run again after first & creating empty files without selecting any.
3. Bugfix: Solved `AttributeError: 'NoneType' object has no attribute 'int'` for yolov8n processed at 87% .
4. Enhancement: Allowed compression and frame extraction if runtime concern is critical.
5. {RENUNCIATED} *Enhancement: Implement a cooler labelling and trajectory (optional enabling) system.
6. Removal: Watermark drawing and bounding boxes' complex color system.
7. Code structure optimizized.

**v0.3 30/11/2023 Submittable**
1. *Bugfix: Integrate DeepSORT to suppress re-ID and duplicate detection problems. <div style="text-align: right"><font color='orange'>2</font></div>
2. Bugfix: Cyclists recognition improved with optical flow.
3. *Bugfix: Enhance the differentiation between parked and mobile vehicles with a distance measuring system. <div style="text-align: right"><font color='red'>1</font></div>
4. {RENUNCIATED} *Bugfix: Suppress non-human object detections in big models (like man on the mailbox ad, sculpture at Peel-Sherbrooke and cars reflected on the glass); augmented missed detections in small models (like the excavator).
5. Added real-time speed estimation using optical flow.
6. Added a buffer mechanism in optical flow calculation to potentially improve efficiency.
7. *Enhancement: Simple GUI with multithreading / batch processing support. <div style="text-align: right"><font color='yellow'>3</font></div>
8. *Enable GPU acceleration: TensorRT, OpenCV with CUDA and more. <div style="text-align: right"><font color='yellow'>3</font></div>
9. *Finetune parameters (search ##). <div style="text-align: right"><font color='red'>1</font></div>
10. Minor code optimizations.
11. As instructed, bus and truck are removed from the car category.
12. Bugfix: Count only passed cars and pedestrians now.

In [18]:
# !pip install ultralytics
# !pip install deep-sort-realtime
# !git clone https://github.com/princeton-vl/RAFT.git

# %cd RAFT
# !.\download_models.sh
# %cd ..

In [23]:
import sys
import cv2
import math
import numpy as np
from collections import defaultdict
from tqdm import tqdm
import threading
from ultralytics import YOLO
import torch
from torchvision.models.optical_flow import Raft_Small_Weights, raft_small
from RAFT.core.utils.utils import InputPadder
from deep_sort_realtime.deepsort_tracker import DeepSort
from PyQt5.QtWidgets import QApplication, QFileDialog

PATH = './Data/'
OUTPUT_VIDEO = True # choose if you need a processed video (saving to PATH) or just statistics (faster)
app = QApplication.instance() or QApplication([])   # QApp should only be created once

In [25]:
def RAFT(frame_prev, frame, roi = .5): #! SEMI-FINISHED. REF[10]
    model = raft_small(weights = Raft_Small_Weights.DEFAULT, progress = False).cuda().half()
    model.eval()
    h, w, _ = frame.shape
    roi = (0, roi * h, w, h)
    _, y1, _, y2 = roi
    frame_prev_roi = frame_prev[y1:y2, :]
    frame_roi = frame[y1:y2, :]
    img_prev = torch.from_numpy(frame_prev_roi).permute(2, 0, 1).float().half().cuda()
    img_curr = torch.from_numpy(frame_roi).permute(2, 0, 1).float().half().cuda()
    img_prev = img_prev[None].cuda()
    img_curr = img_curr[None].cuda()
    padder = InputPadder(img_prev.shape)
    img_prev, img_curr = padder.pad(img_prev, img_curr)
    with torch.no_grad():
        flow_up = model(img_prev, img_curr)
    flow_up = flow_up[-1]
    mag = torch.norm(flow_up, dim = 1).cpu().numpy()
    ang = torch.atan2(flow_up[..., 1], flow_up[..., 0]).cpu().numpy()
    torch.cuda.empty_cache()
    return mag, ang

# REF[4][5][8]
class OF_Buffer:
    def __init__(self, size = 5):
        self.size = size
        self.frames = []
        self.flow_data = []
    def add(self, frame, flow):
        if len(self.frames) >= self.size:
            self.frames.pop(0)
            self.flow_data.pop(0)
        self.frames.append(frame)
        self.flow_data.append(flow)
    def getter(self):   # get the latest frame and its flow data
        if self.frames:
            return self.frames[-1], self.flow_data[-1]
        return None, None

# buffer = OF_Buffer(size = 10)   # depending on available memory. adjust with S, FRAME_PARKED, FRAME_CYCLIST
def OF(frame_prev, frame, gpu = False, roi = .5):   # for GPU, only supports NVIDIA platforms
    if gpu and cv2.cuda.getCudaEnabledDeviceCount():
        g1 = cv2.cuda.cvtColor(cv2.cuda_GpuMat().upload(frame_prev), cv2.COLOR_BGR2GRAY)
        g2 = cv2.cuda.cvtColor(cv2.cuda_GpuMat().upload(frame), cv2.COLOR_BGR2GRAY)
        gpu_flow = cv2.cuda_FarnebackOpticalFlow.create(.5, False, 15, 3, 5, 1.2, 0)
        flow = gpu_flow.calc(g1, g2, None).download()
    else:
        g1 = cv2.cvtColor(frame_prev, cv2.COLOR_BGR2GRAY)
        g2 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        flow = cv2.calcOpticalFlowFarneback(g1, g2, None, .5, 3, 15, 3, 5, 1.2, 0)  ##
    # buffer.add(frame, flow)
    return cv2.cartToPolar(flow[..., 0], flow[..., 1])

def isCyclist(mag, ang):
    if x2 <= x1 or y2 <= y1:
        return False
    mag_box = mag[y1:y2, x1:x2]
    ang_box = ang[y1:y2, x1:x2]
    if mag_box.size == 0 or ang_box.size == 0:
        return False
    avg_flow_hori = np.mean(mag_box * np.sin(ang_box))
    avg_flow_vert = np.mean(mag_box * np.cos(ang_box))
    FLOW_VERT, FLOW_HORI = 1.0, 2.0 ## thresholds for cyclist, featuring relatively linear and large horizontal flow
    return avg_flow_hori > FLOW_HORI and avg_flow_vert < FLOW_VERT

def radar(height, f):  # REF[9]. not used; can help differentiate between parked and mobile vehicles
    """
    Estimate the distance of a vehicle from a monocular camera system
        :param f: Focal length in mm.
        :param height: Real height of the viewpoint vehicle in mm.
        :return: Estimated distance in meters.
    """
    pixel_h = y2 - y1   # apparent height of the detected vehicle in picture
    return (f * height) / pixel_h / 1000    # D = (F * W) / P, converted to meters

def marker(img, lbl = '', status = '', clr = (255, 255, 0)): # REF[6]
    W, H = x2 - x1, y2 - y1
    size = max(.5, min(1.5, .005 * (H * W) ** .5)) * C  # dynamic text relevant to box size (alternatively, use radar())
    L = min(W, H) // 8  # L-shaped corner size
    for (x, y) in [(x1, y1), (x2, y1), (x1, y2), (x2, y2)]:
        cv2.line(img, (x, y), (x + (L if x == x1 else -L), y), clr, int(4 * C))
        cv2.line(img, (x, y), (x, y + (L if y == y1 else -L)), clr, int(4 * C))
    if lbl:
        w, h = cv2.getTextSize(lbl, 0, size, 1)[0]
        coord = y1 - h - 1 if y1 - h - 1 >= h else y1 + h + 1
        cv2.putText(img, lbl, (x1, coord), 0, size, (0, 155, 255), int(2 * C))
    if status:
        w, h = cv2.getTextSize(status, 0, .7 * size, 1)[0]
        x = max(0, min(x2 - w, img.shape[1] - w))
        y = max(h, min(y2 + h + 10, img.shape[0] - h))
        cv2.putText(img, status, (x, y), 0, .7 * size, c, int(2 * C))

In [26]:
C = 1.0                 # image sacling factor. primitve res 2562×1440
S = 1                   # frame extraction freq. effective frame rate 30 / S, starting from the first
assert 0 < C <= 1, "COMPRESSION COEFFICIENT MUST LIE WITHIN (0, 1]"
assert S > 0 and isinstance(S, int), "FRAME SAMPLING INTERVAL MUST BE A POSITIVE INTEGER"
MOVEMENT = 5 * C * S    # pixel distance threshold between frames. requiring dynamic callibration with current speed
PROXIMITY = 100 * C     # proximity threshold between a person and a bike
FRAME_PARKED = 8 // S   # threshold of consecutive frames for a car to be considered parked
FRAME_CYCLIST = 4 // S  # threshold of consecutive frames for a person to be considered cyclist
DELTA = 3.6 * C**2 / S  # correction factor for speed estimation in km/h. REF[5][8]
ROI = .5                # % of which most movements are expected in pictures. primarily varied with dashcam angle
detector = YOLO(PATH + 'yolov8n.pt')    # REF[2]
# tracker = DeepSort(PATH + 'mars-small128.ckpt-68577', nms_max_overlap = .5, max_age = 60, nn_budget = 100)

path, _ = QFileDialog.getOpenFileName(None, "Orion by ThermalG", PATH, "File types (*.mp4 *.avi *.mov)")
if not path:    # REF[7]
    print("NO FILE SELECTED. PROGRAM TERMINATED.")
    sys.exit()
source = cv2.VideoCapture(path)
size = (int(source.get(cv2.CAP_PROP_FRAME_WIDTH) * C), int(source.get(cv2.CAP_PROP_FRAME_HEIGHT) * C))
if OUTPUT_VIDEO:
    fps = int(source.get(cv2.CAP_PROP_FPS)) # primitve FPS 29.97
    # noinspection PyArgumentList
    codec = cv2.VideoWriter_fourcc(*'MP4V') #! X264 can take double storage with compromised output
    output = cv2.VideoWriter(path.rsplit('.', 1)[0] + '_output.mp4', codec, fps / S, size)
v_stack, p_stack, b_stack, cyclist = (defaultdict(list) for _ in range(4))   # tracking history init
parked, v_lastseen, p_lastseen = (defaultdict(int) for _ in range(3))
v_ids, p_ids = {}, {}
summary, speed_trap = [], []
frame_prev = None
ctr = 0 # frame counter
speed = 0

# frame-wise MAIN LOOP
with tqdm(total = math.ceil(source.get(cv2.CAP_PROP_FRAME_COUNT) / S), desc = "PROGRESS", unit = "frame") as pbar:
    while source.isOpened():    # REF[1][3]
        read, frame = source.read()
        if not read:  # read failure (file corrupted) or video end
            break
        if ctr % S == 0:
            frame = cv2.resize(frame, size)
            results = detector.track(frame, conf = .5, persist = True, verbose = False) ##
            if frame_prev is not None:  # REF[4][5]
                mag, ang = OF(frame_prev, frame)
                ## weight-average for smoothing at the cost of latency in showing speed change (not affecting max)
                speed = .9 * speed + .1 * DELTA * np.mean(mag[int(ROI * mag.shape[0]):])
                speed_trap.append(speed)
                if OUTPUT_VIDEO:
                    cv2.putText(frame, f'Speed: {speed:.1f} KPH', (int(.05 * size[0]), int(.05 * size[1])), 2, 2 * C, (255, 0, 127), int(2 * C))
            if results[0].boxes is not None and results[0].boxes.id is not None:
                for id, box in zip(results[0].boxes.id.int().cpu().tolist(), results[0].boxes.data):
                    x1, y1, x2, y2 = map(int, box[:4])
                    idx = int(box[-1])
                    if idx == 2:    # car category
                        v_lastseen[id] = ctr
                        if id not in v_ids:
                            v_ids[id] = len(v_ids)
                        pos = (box[0], box[1])
                        pos_prev = v_stack[id][-1] if id in v_stack else pos
                        v_stack[id].append(pos)
                        # monitor vehicle movement
                        dist = math.hypot(abs(pos_prev[0] - pos[0]) + abs(pos_prev[1] - pos[1]))
                        ## subjective car in st-catherines is faster and closer to parked cars so higher threshold
                        diff = speed * MOVEMENT / DELTA * (1 if path == PATH + "mcgill_drive.mp4" else 8)
                        if dist < 1.5 * diff:
                            if dist < 0.05 * diff:
                                status = 'INDECISIVE'  # mobile in the same lane ahead or parked far away. not passed so not interested nor counted
                            else:
                                parked[id] += 1
                                status = 'PARKED' if parked[id] > FRAME_PARKED else 'MOBILE'
                        else:
                            parked[id] = 0
                            status = 'MOBILE'
                        c = (0, 0, 255) if status == 'MOBILE' else (0, 255, 0) if status == 'PARKED' else (0, 0, 0)
                        if OUTPUT_VIDEO:
                            marker(frame, f'CAR{[idx]} {v_ids[id] + 1}', status, c)
                    elif idx == 0 and frame_prev is not None:   # person category
                        p_lastseen[id] = ctr
                        if isCyclist(mag, ang): # rule out cyclists - auxiliary algo
                            continue
                        pos = (box[0], box[1])
                        p_stack[id].append(pos)
                        if id not in p_ids and id not in cyclist:
                            p_ids[id] = len(p_ids)
                        if OUTPUT_VIDEO:
                            marker(frame, f'PEDESTRIAN {p_ids[id] + 1}')
                    elif idx in [1, 3]:     # bicycle and motorcycle categories
                        b_stack[id].append((box[0], box[1]))
                for b_id, p_pos in p_stack.items(): # rule out cyclists - main algo
                    for b_pos in b_stack.items():   # REF[6]
                        if len(b_pos) >= FRAME_CYCLIST and len(p_pos) >= FRAME_CYCLIST and all(abs(p_pos[-i][0] - b_pos[-i][0].item()) + abs(p_pos[-i][1] - b_pos[-i][1].item()) < PROXIMITY for i in range(1, FRAME_CYCLIST + 1)):
                            cyclist.add(b_id)
                            break
            # update counts
            summary = [
                f'MOBILE: {max(0, len([v for v, f in v_lastseen.items() if f < ctr]) - sum(parked[v] > FRAME_PARKED for v in parked))}',
                f'PARKED: {sum(parked[v] > FRAME_PARKED for v in parked)}',
                f'PEDESTRIANS: {len([p for p, f in p_lastseen.items() if f < ctr]) - len(cyclist)}'
            ]
            if OUTPUT_VIDEO:
                for i, info in enumerate(summary, start = 1):
                    cv2.putText(frame, info, (int(.8 * size[0]), int(.03 * i * size[1])), 0, C, (255, 0, 0), int(2 * C))
                # noinspection PyUnboundLocalVariable
                output.write(frame)
            pbar.update(1)  # param other than 1 requires extra check, pointless
            frame_prev = frame.copy()   # previous frame now becomes current frame
        ctr += 1

print('\n'.join(summary))
print(f'Interval Maximum Speed: {max(speed_trap):.1f} KPH')
source.release()
if OUTPUT_VIDEO:
    output.release()
    cv2.destroyAllWindows()

PROGRESS: 100%|██████████| 1071/1071 [12:12<00:00,  1.46frame/s]


MOBILE PASSED: 24
PARKED PASSED: 20
PEDESTRIANS PASSED: 26
Interval Maximum Speed: 39.0 KPH


PROGRESS: 100%|██████████| 1477/1477 [20:22<00:00,  1.21frame/s]

MOBILE PASSED: 8
PARKED PASSED: 60
PEDESTRIANS PASSED: 67
Interval Maximum Speed: 47.2 KPH


<center><h2>PROJECT REPORT</h2></center>

#### Introduction - Procecure Overview
The ORION system is designed to analyze dashcam videos to provide detailed analytics on moving and parked cars, as well as pedestrian traffic. It employs advanced computer vision techniques, including YOLO for object detection and RAFT for optical flow calculations. The project aims to offer insights into traffic patterns and potentially improve dashcam technology for practical applications like traffic management and driver assistance systems.

&nbsp;
#### Main - Performance Evaluation
The system was evaluated against manually obtained ground truth values for two dashcam videos.
&nbsp;**Ground Truth** (very unlikely accurate, especially pedestrians in st-catherines_drive.mp4)

| counts        | mcgill_drive | st-catherines_drive |
|---------------|--------------|---------------------|
| mobile passed | 24           | 0                   |
| parked passed | 16           | 59                  |
| pedestrians   | 35           | 108                 |

**Predicted**

| counts        | mcgill_drive | st-catherines_drive |
|---------------|--------------|---------------------|
| mobile passed | 24           | 8                   |
| parked passed | 20           | 60                  |
| pedestrians   | 26           | 67                  |

For speed estimation, please refer to the printed statistics above directly. My eyes are not speedometer so ground truth seems not possible.

**Efficiency**: Currently the efficiency of the system is quite low. The majority of computational stress lies in calculating optical flow (with CPU now, GPU idling most of the time), which is only used for the optional functionality - speed estimation in the current version. Eliminating real-time speed estimation can reduce the processing time by 10 times (to around 12 frames/s). Further, if not interested in the video output (debugging purpose) and just to meet the assignment requirements, we can set OUTPUT_VIDEO to False, further increasing speed to 18 frames/s on 1650S + 12600K. Currently we observed that even without optical flow, the avg GPU usage is only 30%. Finishing the acceleration can easily boost above 30 frames/s on any middle-end GPU to achieve real-time processing.

**Accuracy**: We dedicated most effort to accuracy details: exluding cyclists, differentiating parked and mobile cars, avoiding re-ID and duplication, etc. In the tables above, you can see our system is able to achieve 90% accuracy on the given videos. However, it is not robust enough to handle different traffic scenarios. This is because MOVEMENT is constant for videos with different speed and traffic condition. Also, you may have noticed some cyclists are still detected occasionally. This is probably due to the relatively poor detection ability of the yolov8n model we chose. In these frames, the bicycles, which the exclusion algo relies on, are not detected because they are of irregualr shapes or massively blocked by the cyclists. Last, the biggest disparency is that predicted pedestrian count always seems less. Such cpnsequences are very much expected because even when we were determining ground truth with shrewd eyes, some person objects are so secluded. Using larger models instead of nano can improve the situation, but lead to other more intractable problems, like detecting reflected objects on glass, non-person objects like the sculpture at Peel-Sherbrooke. However, nano does have its own glitches. For instance, it recognizes man on the mailbox poster and misses the excavator. But balancing with runtime, nano proves worthy. We believe these problems can be solved by utilizing our optical flow algo.

**Robustness**: Robustness has been improved to handle all video file types and all kinds of anticipated bugs during processing (like if no objects detected in a frame). We also integrated a frame extraction and video compression mechnism to speed up the processing if needed. There is no need to waste words on elaborating this part. You can refer to our change log at the beginning.

**Cross-Nominal Comparison**: After reviewing some similar road monitor system on github and youtube, we are confident that our model is doing the job quite well.

&nbsp;
#### Overall Approach
The program aims to detect, track, and classify cars and pedestrians (trucks, buses, cyclists can be easily added), and calculate the subjective car's speed. Below is the methodology walkthrough.

1. Deep Learning for Object Detection and Tracking
Utilizes the YOLO (You Only Look Once) deep learning model for real-time object detection.
Applies DeepSort for tracking objects across frames (yet to be integrated).

2. Optical Flow for Motion Analysis
Implements two methods for calculating optical flow (OF_NV and OF), which is a technique to estimate motion between video frames.
The OF_NV method uses the RAFT (Recurrent All-Pairs Field Transforms) model for high-quality flow estimation and an alternative implementation using CUDA for GPU acceleration.
The OF method is a more traditional approach using cv2.calcOpticalFlowFarneback. This is the default method used in this assignment and is the slowest.

3. Data Processing and Analysis
Frames are processed at a specific frequency, determined by the S variable.
For each frame, object detection is performed to identify cars, pedestrians, bicycles, etc.
Optical flow data is used to analyze motion and compute the vehicle's speed.

4. Specific Object Detection and Behavior Analysis
The code differentiates between cars, pedestrians, and cyclists using both detection labels and motion patterns.
It identifies parked cars by tracking their movement over time and checking if the movement is below a certain threshold.
Cyclists are identified using a combination of detection labels and movement patterns that are characteristic of cycling.

5. Speed Estimation
The vehicle's speed is estimated using optical flow data. This involves calculating the average flow of pixels in a specified region of interest and applying a correction factor for accuracy.

6. Visual Annotations and Output
The code can optionally output a processed video with visual annotations, like bounding boxes, labels (e.g., 'CAR', 'PEDESTRIAN'), and the vehicle's speed.
It uses functions like cv2.putText and cv2.line to draw on the video frames.

7. Data Structures for Tracking and Analysis
Uses dictionaries and lists to store tracking data and history for each detected object.
Maintains counts of different types of objects and their states (e.g., parked vs. mobile cars).

8. User Interface and File Handling (commented out)
The code includes a segment for file selection through a GUI using QFileDialog.
Handles video files, reading frames from them, and potentially saving the processed output.

&nbsp;
**Assumptions**
Video Source: Assumes the input is video footage, likely from a vehicle-mounted camera.
CUDA Support: Assumes the availability of CUDA-enabled devices for GPU acceleration.
Environment Setup: Assumes a Python environment with necessary dependencies installed.
File Paths and Formats: Assumes specific file paths and formats for model weights and video files.
GUI Components: Assumes the use of a graphical user interface for file selection.

&nbsp;
#### Description of Used Software Package & Routine
Ultralytics YOLO: A deep learning model for real-time object detection. It's used here to detect objects like cars, pedestrians, and cyclists in each video frame.
DeepSORT: A real-time tracking algorithm that uses deep learning features to track detected objects across frames.
RAFT (Recurrent All-Pairs Field Transforms): A deep learning model for optical flow prediction. It's used to estimate the motion between two image frames.
PyQt5: A set of Python bindings for Qt application framework, used for GUI elements like file dialogs.
TQDM: A library providing progress bars for loops and other iterative processes.
*The rest of packages imported are rather common. No need to elaborate.*

&nbsp;
#### Conclusion - Future Work
The ORION system demonstrates promising capabilities in traffic analytics using computer vision. However, it is semi-finished and our future work can focus on:

Improving Accuracy: Enhancing the model's ability to differentiate between closely spaced objects and in diverse lumiantion conditions. This means multiple optimizations to the current model, especially finishing the integration of DeepSORT to suppress re-ID and duplicate detection problems.
Real-Time Processing: Finish the GPU acceleration algorithms for faster processing to support real-time analytics.
Extended Functionality: Finish the distance measuring system as an auxiliary algorithm to differentiate between parked and mobile vehicles.
User Interface: Curently we comment out the GUI for video selecting and hardcoded it to just process the two interested videos, since this is an assignment so we should make it grading-friendly in one notebook. In the future, we might want to develop an intuitive interface for users to interact with the system and access analytics more efficiently.

&nbsp;
## References
1. **Zhao, C.** (2023, September 14). *YOLOv8 based target tracking - car tracking and counting*.
    - [CSDN](https://blog.csdn.net/zhaocj/article/details/132800296)
    - accessed: 19/11/2023
2. **Ultralytics YOLOv8 Docs**. (n.d.). *Multi-Object Tracking with Ultralytics YOLO*.
    - [Ultralytics](https://docs.ultralytics.com/modes/track/)
    - accessed: 19/11/2023
3. **Moin, M.** (2023, January 25). *YOLOv8-DeepSORT-Object-Tracking*.
    - [GitHub](https://github.com/MuhammadMoinFaisal/YOLOv8-DeepSORT-Object-Tracking)
    - accessed: 21/11/2023
4. **Kuklin, M.** (2021, July 16). *Optical Flow in OpenCV (C++/Python)*.
    - [LearnOpenCV](https://learnopencv.com/optical-flow-in-opencv/)
    - accessed: 22/11/2023
5. **Shafu0x**. (n.d.). *Vehicle Speed Estimation from Video using Deep Learning and Optical Flow in PyTorch*.
    - [GitHub](https://github.com/shafu0x/vehicle-speed-estimation)
    - accessed: 22/11/2023
6. **How to detect if object is stationary OpenCV**. (n.d.).
    - [Stack Overflow](https://stackoverflow.com/questions/68683164/how-to-detect-if-object-is-stationary-opencv)
    - accessed: 22/11/2023
7. **PyQT Documentation v5.15.4**. (n.d.).
    - [QTWidgets](https://www.riverbankcomputing.com/static/Docs/PyQt5/api/qtwidgets/qtwidgets-module.html)
    - accessed: 20/11/2023
8. **Object for estimating optical flow using Farneback method**. (n.d.).
    - [MATLAB](https://www.mathworks.com/help/vision/ref/opticalflowfarneback.html)
    - accessed: 23/11/2023
9. **yolov8-车辆测距+追尾预警+车辆检测识别+车辆跟踪测速**。 (n.d.). 
    - [CSDN](https://blog.csdn.net/weixin_44944382/article/details/128518452)
    - accessed: 26/11/2023
10. **Teed, Z., & Deng, J.** (2020). Raft: Recurrent all-pairs field transforms for optical flow. In *Computer Vision–ECCV 2020: 16th European Conference, Glasgow, UK, August 23–28, 2020, Proceedings, Part II 16* (pp. 402-419). Springer International Publishing.